# Portfolio Optimization 
Select a value for $\xi$ and solve the program
$$
\begin{equation}
    \min_{\{w_{i}\}_{i \in \{1, 2,..., K\}}} [-E(R) R_{B} + \xi \mathrm{VAR}(R)]
\end{equation}
$$

to select a weighted proportion of the portfolio to seek maximum return and minimized risk.

In [1]:
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from eqc_models.allocation import PortMomentum
from eqc_models.solvers import Dirac3ContinuousCloudSolver
from utils import (
    get_nasdaq100_constituents,
    get_port_stats,
)

In [3]:
# Set parameters
ADJ_DATE = "2022-01-01"
LOOKBACK_DAYS = 60
LOOKFORWARD_DAYS = 30

# Get stock list
stocks = get_nasdaq100_constituents(
    ADJ_DATE, LOOKBACK_DAYS, LOOKFORWARD_DAYS,
)

# Get portfolio model
model = PortMomentum(
    stocks=stocks,
    adj_date=ADJ_DATE,
    stock_data_dir="stock_prices",
    lookback_days=LOOKBACK_DAYS,
    window_days=30,
    window_overlap_days=15,
    weight_upper_limit=0.08,
    r_base=0.05 / 365,
    alpha=5.0, # penalty multiplier term
    beta=1.0, # penalty multiplier term
    xi=1.0, # multiplier as in the formula
)

Chose 101 of 102 stocks


In [4]:
# Solve on Dirac-3
solver = Dirac3ContinuousCloudSolver()
response = solver.solve(
    model,
    sum_constraint=100,
    relaxation_schedule=2,
    solution_precision=None,
    num_samples=10,
)
sol = response["results"]["solutions"][0][:len(stocks)]

print(response)

weight_hash = {}
for i in range(len(stocks)):
    weight_hash[stocks[i]] = sol[i] / 100.0

tot_weight = sum(weight_hash.values())

if tot_weight != 1.0:
    for stock in stocks:
        weight_hash[stock] = weight_hash[stock] / tot_weight

weight_df = pd.DataFrame(
    {
        "Stock": [item for item in weight_hash.keys()],
        "Allocation": [
            weight_hash[item] for item in weight_hash.keys()
        ],
    }
)
weight_df["Date"] = ADJ_DATE
weight_df = weight_df[weight_df["Allocation"] > 0]

ret_df = get_port_stats(weight_df, 30)

print(ret_df)


DEBUG:eqc_models.solvers.qciclient:Getting QciClient connection using environment variables
DEBUG:eqc_models.solvers.qciclient:Min degree of polynomial 1
DEBUG:eqc_models.solvers.qciclient:Max degree of polynomial 2
DEBUG:eqc_models.solvers.qciclient:Number of polynomial elements 5555
DEBUG:eqc_models.solvers.qciclient:{'file_name': 'PortMomentum', 'file_config': {'polynomial': {'num_variables': 202, 'max_degree': 2, 'min_degree': 1, 'data': [{'idx': [0, 1], 'val': -1016.0}, {'idx': [0, 2], 'val': -1016.0}, {'idx': [0, 3], 'val': -1016.0}, {'idx': [0, 4], 'val': -1016.0}, {'idx': [0, 5], 'val': -1016.0}, {'idx': [0, 6], 'val': -1016.0}, {'idx': [0, 7], 'val': -1016.0}, {'idx': [0, 8], 'val': -1016.0}, {'idx': [0, 9], 'val': -1016.0}, {'idx': [0, 10], 'val': -1016.0}, {'idx': [0, 11], 'val': -1016.0}, {'idx': [0, 12], 'val': -1016.0}, {'idx': [0, 13], 'val': -1016.0}, {'idx': [0, 14], 'val': -1016.0}, {'idx': [0, 15], 'val': -1016.0}, {'idx': [0, 16], 'val': -1016.0}, {'idx': [0, 17], '

2025-03-14 23:09:08 - Dirac allocation balance = 0 s (unmetered)


DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "POST /optimization/v1/jobs HTTP/1.1" 201 37
DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/jobs/67d50b7400e804f113aefcaf/status HTTP/1.1" 200 107


2025-03-14 23:09:09 - Job submitted: job_id='67d50b7400e804f113aefcaf'
2025-03-14 23:09:09 - QUEUED


DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/jobs/67d50b7400e804f113aefcaf/status HTTP/1.1" 200 108


...

2025-03-14 23:12:21 - COMPLETED


DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/jobs/allocations HTTP/1.1" 200 121
DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/jobs/67d50b7400e804f113aefcaf HTTP/1.1" 200 576
DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/jobs/67d50b7400e804f113aefcaf/status HTTP/1.1" 200 110
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): mcampanelli-23481-0616.aws.qci-dev.com:443


2025-03-14 23:12:23 - Dirac allocation balance = 0 s (unmetered)


DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/files/67d50c32236cc2225ac90c78 HTTP/1.1" 200 233
DEBUG:urllib3.connectionpool:https://mcampanelli-23481-0616.aws.qci-dev.com:443 "GET /optimization/v1/files/67d50c32236cc2225ac90c78/contents/1 HTTP/1.1" 200 None


{'job_info': {'job_id': '67d50b7400e804f113aefcaf', 'job_submission': {'problem_config': {'normalized_qudit_hamiltonian_optimization': {'polynomial_file_id': '67d50b72236cc2225ac90c74'}}, 'device_config': {'dirac-3_normalized_qudit': {'num_samples': 10, 'relaxation_schedule': 2, 'sum_constraint': 100}}}, 'job_status': {'submitted_at_rfc3339nano': '2025-03-15T05:09:08.796Z', 'queued_at_rfc3339nano': '2025-03-15T05:09:08.797Z', 'running_at_rfc3339nano': '2025-03-15T05:09:08.906Z', 'completed_at_rfc3339nano': '2025-03-15T05:12:18.785Z'}, 'job_result': {'file_id': '67d50c32236cc2225ac90c78', 'device_usage_s': 82}}, 'status': 'COMPLETED', 'results': {'counts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'energies': [-51499.1796875, -51499.1015625, -51498.4101562, -51498.03125, -51498.0117188, -51497.875, -51497.1835938, -51497, -51496.109375, -51495.9921875], 'solutions': [[1.0337895, 1.041032, 0.9009118, 0.972897, 1.014949, 0.9984183, 1.016353, 1.0429311, 0.9895242, 0.9975287, 0.9624027, 0.9402848, 0.